In [ ]:
import scipy.io
import scipy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from IPython.display import display

from sklearn.feature_selection import mutual_info_regression as MIR


import torch, torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F

import datetime
import pickle


In [ ]:
def calc_data_sent(t_0, t_i, gh, gt, D, W, eff, P_AP, N0, p_max):
    energy_harvested = t_0 * eff * P_AP * gh
    power_to_comsume_energy = energy_harvested / t_i
    power_to_comsume_energy[power_to_comsume_energy > p_max] = p_max
    data_sent = t_i * W * np.log2(1 + (power_to_comsume_energy * gt) / (N0 * W))
    return data_sent

In [ ]:
def outage_calc(t_0_dim, t_i, gh, gt, D, W, eff, P_AP, N0, p_max):
    t_0 = t_0_dim.reshape(len(t_0_dim), 1)
    energy_harvested = t_0 * eff * P_AP * gh
    power_to_comsume_energy = energy_harvested / t_i
    power_to_comsume_energy[power_to_comsume_energy > p_max] = p_max
    data_sent = t_i * W * torch.log2(1 + (power_to_comsume_energy * gt) / (N0 * W))
    outage = D - data_sent
    # outage[outage<0]=0
    outage = torch.abs(outage)
    return torch.sum(outage)

In [ ]:
def loss_fn(t_0, output_time, tgt_time, gh, gt):
    mse_loss = nn.MSELoss()
    outage = outage_calc(
        t_0, output_time / scale_factor, gh, gt, D, W, eff, P_AP, N0, p_max
    )
    mse = mse_loss(output_time, tgt_time)
    return 10 * outage, outage

In [ ]:
input_data_folder = '../input_data'
output_data_folder = '../output/sub_dnn_models/'
tensor_board_folder = '../runs/sub_dnn'

In [ ]:
## Fixed params
eff = 0.5
nu = 1
PL_d0_dB = 31.67
alpha = 2
Z_stdDev = 2
W = 1e6
N0 = 1e-14
D = 50

scale_factor = 1e7
numChReal = 1000000
## params changing based on input file
P_AP = 2
p_max = 0.01
num_of_user = 4
train_data_size=100

In [ ]:
##  LOAD DATA
match = scipy.io.loadmat(input_data_folder + "/channel_gains.mat")
test_ind=np.load(input_data_folder+'/test_ind.npy')

In [ ]:
feat_dict={}
for num_of_user in range(2,5,2):
    val_loss_list = []
    ## DNN Arch  ##
    class SubNet(nn.Module):

        def __init__(self):
            super(SubNet, self).__init__()
            self.layer1 = nn.Linear(3 * num_of_user + 1, 8 * num_of_user)
            self.layer2 = nn.Linear(8 * num_of_user, 8 * num_of_user)
            self.layer3 = nn.Linear(8 * num_of_user, 8 * num_of_user)
            self.layer4 = nn.Linear(8 * num_of_user, 4 * num_of_user)
            self.layer5 = nn.Linear(4 * num_of_user, 4 * num_of_user)
            self.output = nn.Linear(4 * num_of_user, num_of_user)

        def forward(self, input_x):
            x = F.relu(self.layer1(input_x))
            x = F.relu(self.layer2(x))
            x = F.relu(self.layer3(x))
            x = F.relu(self.layer4(x))
            x = F.relu(self.layer5(x))

            x = torch.abs(self.output(x))
            return x

    ## Data Load
    mat = scipy.io.loadmat(
        input_data_folder
        + "/v2_subdnn_label_N014"
        + "_PAP"
        + str(P_AP)
        + "_M"
        + str(num_of_user)
        + "_pmax"
        + str(p_max)
        + ".mat"
    )

    gh = match["gh_arr"][0:num_of_user, 0:numChReal]
    gt = match["gt_arr"][0:num_of_user, 0:numChReal]

    gamma = gh * gt * eff * P_AP / (W * N0)
    alpha = np.abs(scipy.special.lambertw(np.exp(-1) * (gamma - 1), k=0) + 1)

    df = pd.DataFrame(mat["label_time_for_sub_dnn"])
    df["ind"] = mat["ind_arr"][:, 0]
    # df2=pd.DataFrame(np.concatenate((gh,gt,alpha,gamma,beta,theta)).T)
    df2 = pd.DataFrame(np.concatenate((gh, gt, alpha, gamma)).T)
    df2.set_index(df2.index.values + 1)
    df = df.merge(df2, right_index=True, left_on="ind")
    df_test = df.loc[df["ind"].isin(test_ind)]
    df_train = df.loc[~df["ind"].isin(test_ind)]

    input_columns = [0] + list(np.arange(num_of_user + 2, df.shape[1]))
    output_columns = np.arange(1, num_of_user + 1)

    X_test = df_test.iloc[:, input_columns].values
    y_test = df_test.iloc[:, output_columns].values
    y_test = y_test * scale_factor
    X_train = df_train.iloc[:, input_columns].values
    y_train = df_train.iloc[:, output_columns].values
    y_train = y_train * scale_factor

    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.01, random_state=42
    )

    # Scale
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_val = ss.transform(X_val)
    X_test = ss.transform(X_test)
    pickle.dump(ss, open(output_data_folder+'scaler_N'+str(num_of_user)+'.pkl','wb'))

    # Feature Selection
    mi = MIR(X_train[0:10000, 1:], y_train[0:10000].sum(axis=1))
    feature_index = np.argsort(mi)
    feat_size = 3 * num_of_user

    feature_index = np.argsort(mi)
    feature_index = [0] + list(feature_index[-1 * feat_size :] + 1)
    feat_dict[num_of_user]=feature_index
    

    # Dataset & DataLoder
    torch_dataset = TensorDataset(
        torch.tensor(X_train[0:train_data_size, :].astype(np.float32)),
        torch.tensor(y_train[0:train_data_size, :].astype(np.float32)),
    )
    val_torch_dataset = TensorDataset(
        torch.tensor(X_val.astype(np.float32)), torch.tensor(y_val.astype(np.float32))
    )

    train_data_loader = DataLoader(torch_dataset, batch_size=32)
    val_data_loader = DataLoader(val_torch_dataset, batch_size=1)

    ## TRAINING
    # default `log_dir` is "runs" - we'll be more specific here
    writer = SummaryWriter(
        tensor_board_folder
        + "/trainsize"
        + str(train_data_size)
        + "_M"
        + str(num_of_user)
        + "_PAP"
        + str(P_AP)
        + "_pmax"
        + str(p_max)
    )
    NO_EPOCHS = 20

    val_loss_best = 10000000

    model = SubNet()
    optimizer = Adam(model.parameters(), lr=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, "min")
    loss_fn2 = nn.MSELoss()
    for epoch_idx in range(NO_EPOCHS):
        model.train()
        epoch_loss = 0
        len_batches = 0
        for ii, sample in enumerate(tqdm(train_data_loader)):
            local_inp, local_tgt = sample
            local_inp_inv = torch.Tensor(ss.inverse_transform(local_inp))
            output = model(local_inp[:, feature_index])
            # local_inp_inv = torch.Tensor(ss.inverse_transform(local_inp))
            # output = model(local_inp[:,8:],local_inp_inv)
            # loss = loss_fn(output,local_tgt)
            [loss, outage] = loss_fn(
                local_inp_inv[:, 0],
                output,
                local_tgt,
                local_inp_inv[:, 1 : num_of_user + 1],
                local_inp_inv[:, num_of_user + 1 : 2 * num_of_user + 1],
            )

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.detach().numpy()

        # Validation phase
        model.eval()
        val_loss = 0
        val_outage = 0

        with torch.no_grad():
            for jj, val_sample in enumerate(val_data_loader):
                local_inp, local_tgt = val_sample
                local_inp_inv = torch.Tensor(ss.inverse_transform(local_inp))
                output = model(local_inp[:, feature_index])
                # output = model(local_inp[:,8:],local_inp_inv)
                val_loss += loss_fn2(output, local_tgt)
                [temp_loss, val_outage_temp] = loss_fn(
                    local_inp_inv[:, 0],
                    output,
                    local_tgt,
                    local_inp_inv[:, 1 : num_of_user + 1],
                    local_inp_inv[:, num_of_user + 1 : 2 * num_of_user + 1],
                )
                # val_loss += temp_loss
                val_outage += val_outage_temp

            val_loss /= jj + 1
            val_outage /= jj + 1

        if val_loss < val_loss_best:
            val_loss_best = val_loss
            torch.save(model, output_data_folder+'/model_N'+str(num_of_user)+'.pt')
            # X_test_inverse = torch.Tensor(ss.inverse_transform(X_test))
            # test_output = model((torch.tensor(X_test.astype(np.float32))),X_test_inverse)

        # writer.add_scalar('val_loss', val_loss, epoch_idx)

        # writer.add_scalar('training loss', epoch_loss / (ii+1),epoch_idx)
        # writer.add_scalar('val_outage',val_outage,epoch_idx)
        # writer.add_scalar('train_outage',outage / (ii+1),epoch_idx)
    val_loss_list.append(val_loss)
np.save(
    output_data_folder+"/sub_dnn", val_loss_list
)

In [ ]:
def fixed_point_iteration(M, D, W, gamma, t0, err_tol):
    tau = np.zeros(M)
    for i in range(0, M):
        ti_prev = 0
        ti = 10
        while np.abs(ti_prev - ti) > err_tol * 1e-3:
            ti_prev = ti
            ti = D[i] / (W * np.log2(1 + gamma[i] * (t0 / ti_prev)))
        tau[i] = ti
    return tau


def bisectionSearch_dnn(
    N,
    betaThr,
    C_constants,
    lb,
    ub,
    errTol,
    D,
    gamma,
    W,
    dnn_ins,
    std_scaler,
    outpt_scale_factor,
    dnn_model,
):

    insTau_0 = ub
    tau = C_constants

    while ub - lb > 2 * errTol:
        insTau_0 = (ub + lb) / 2
        ins = np.append(insTau_0, dnn_ins)
        ins = std_scaler.transform(ins.reshape(1, len(ins)))
        ins = ins[0, feat_dict[num_of_user]].reshape(1, 3 * num_of_user + 1)
        dnn_out = model((torch.tensor(ins.astype(np.float32))))
        dnn_out = dnn_out / scale_factor
        tau = dnn_out.detach().numpy()[0]

        # tau= fixed_point_iteration(N,D,W,gamma,insTau_0,errTol);
        index = insTau_0 >= betaThr
        tau[index] = C_constants[index]

        # print('******')
        # print(tau)
        # print(tau2)

        turev = 1
        for i in range(0, N):
            if insTau_0 < betaThr[i]:
                turev = turev + gamma[i] * tau[i] * W / (
                    2 ** (D[i] / (W * tau[i])) * W * tau[i]
                    - W * tau[i]
                    - 2 ** (D[i] / (W * tau[i])) * D[i] * np.log(2)
                )
        if turev >= 0:
            ub = insTau_0
            # print('ub updated')
        if (turev <= 0) or (np.isnan(turev)):
            lb = insTau_0

    optimalEHPoint = insTau_0

    return optimalEHPoint, tau


def MRTTMA_dnn(
    K,
    N,
    D,
    p_max,
    eff,
    N0,
    P_AP,
    W,
    g_harvesting,
    g_transmission,
    errTol,
    std_scaler,
    outpt_scale_factor,
    dnn_model,
):
    # g_transmission=g_transmission.T[0]
    # g_harvesting=g_harvesting.T[0]
    gamma = g_harvesting * g_transmission * eff * P_AP / (W * N0)
    alpha = np.abs(scipy.special.lambertw(np.exp(-1) * (gamma - 1), k=0) + 1)

    theta = D / (W * np.log2(1 + (p_max * g_transmission) / (W * N0)))  # ti at pmax
    beta = theta * p_max / (g_harvesting * eff * P_AP)  # t0 at pmax

    # C_constants=D / (W * np.log2((p_max * gamma)/ (P_AP * eff * g_harvesting) + 1));
    # beta = (p_max * D) / (W*eff*P_AP* g_harvesting * np.log2((p_max * g_transmission)/ (N0*W) + 1));

    ins = np.concatenate((g_harvesting, g_transmission, alpha, gamma))

    lb = 0
    ub = np.max(beta)

    # gamma=gamma.reshape(K+N,1)
    t0_opt, tau = bisectionSearch_dnn(
        K + N,
        beta,
        theta,
        lb,
        ub,
        errTol * 1e-2,
        D,
        gamma,
        W,
        ins,
        std_scaler,
        outpt_scale_factor,
        dnn_model,
    )
    # energy_harvested=g_harvesting * eff * P_AP * t0_opt
    # tx_power=energy_harvested/ tau
    # tx_power[tx_power<1e-2]=1e-2
    # data_sent=tau*W*np.log2(1+(tx_power*g_transmission)/(W*N0))
    return t0_opt, tau

In [ ]:
avg_sub_dnn_runtime = []
min_sub_dnn_runtime = []
max_sub_dnn_runtime = []
avg_sub_dnn_tottime = []
for num_of_user in range(2, 5, 2):
    print(num_of_user)
    # mat = scipy.io.loadmat(input_data_folder+'/subdnn_label_N014'+'_PAP'+str(P_AP)+'_M'+str(num_of_user)+'_pmax'+str(p_max)+'.mat')

    gh = match["gh_arr"][0:num_of_user, 0:numChReal]
    gt = match["gt_arr"][0:num_of_user, 0:numChReal]

    gh_test = match["gh_arr"][0:num_of_user, test_ind].T
    gt_test = match["gt_arr"][0:num_of_user, test_ind].T

    model = torch.load(output_data_folder + "model_N" + str(num_of_user) + ".pt")
    model.eval()
    ss = pickle.load(
        open(output_data_folder + "scaler_N" + str(num_of_user) + ".pkl", "rb")
    )

    sub_dnn_runtime = np.zeros(len(test_ind))
    sub_dnn_tottime = np.zeros(len(test_ind))
    gh_test = match["gh_arr"][0:num_of_user, test_ind].T
    gt_test = match["gt_arr"][0:num_of_user, test_ind].T
    for ind in range(len(test_ind)):
        time_start = datetime.datetime.now()
        t0_opt, tau = MRTTMA_dnn(
            0,
            num_of_user,
            D * np.ones(num_of_user),
            p_max,
            eff,
            N0,
            P_AP,
            W,
            gh_test[ind, :],
            gt_test[ind, :],
            1e-8,
            ss,
            scale_factor,
            model,
        )
        time_stop = datetime.datetime.now()
        data_sent_arr = calc_data_sent(
            t0_opt, tau, gh_test[ind, :], gt_test[ind, :], D, W, eff, P_AP, N0, p_max
        )
        num_time_frame3 = D / data_sent_arr
        num_time_frame3[num_time_frame3 < 1] = 1
        sub_dnn_tottime[ind] = (
            np.max(num_time_frame3) * t0_opt + np.multiply(num_time_frame3, tau).sum()
        )
        # sub_dnn_tottime[ind]=t0_opt+tau.sum()
        sub_dnn_runtime[ind] = (time_stop - time_start).total_seconds()
    avg_sub_dnn_runtime.append(np.mean(sub_dnn_runtime))
    min_sub_dnn_runtime.append(np.min(sub_dnn_runtime))
    max_sub_dnn_runtime.append(np.max(sub_dnn_runtime))
    avg_sub_dnn_tottime.append(np.mean(sub_dnn_tottime))
#np.savez("runtime_results/xai_all_trainsize" + str(train_data_size) + "subdnn_runtime.npz",mean=avg_sub_dnn_runtime,min=min_sub_dnn_runtime,max=max_sub_dnn_runtime,)
# np.save('sub_dnn_models/xai_trainsize'+str(train_data_size)+'all_tot_times_N014_PAP2_M'+str(num_of_user)+'_pmax0.01.npy',avg_sub_dnn_tottime)

In [ ]:
class SubNet(nn.Module):

    def __init__(self):
        super(SubNet, self).__init__()
        self.layer1 = nn.Linear(4 * num_of_user + 1, 8 * num_of_user)
        self.layer2 = nn.Linear(8 * num_of_user, 8 * num_of_user)
        self.layer3 = nn.Linear(8 * num_of_user, 8 * num_of_user)
        self.layer4 = nn.Linear(8 * num_of_user, 4 * num_of_user)
        self.layer5 = nn.Linear(4 * num_of_user, 4 * num_of_user)
        self.output = nn.Linear(4 * num_of_user, num_of_user)

        self.droplayer1 = nn.Dropout(0.5)
        self.droplayer2 = nn.Dropout(0.5)
        self.droplayer3 = nn.Dropout(0.5)
        self.droplayer4 = nn.Dropout(0.5)
        self.droplayer5 = nn.Dropout(0.5)

    def forward(self, input_x):
        x = F.relu(self.layer1(input_x))
        # x=self.droplayer1(x)
        x = F.relu(self.layer2(x))
        # x=self.droplayer2(x)
        x = F.relu(self.layer3(x))
        # x=self.droplayer3(x)
        x = F.relu(self.layer4(x))
        # x=self.droplayer4(x)
        x = F.relu(self.layer5(x))
        # x=self.droplayer5(x)

        x = torch.abs(self.output(x))
        return x